<a href="https://colab.research.google.com/github/mengmengwoo/CNN-LSTM-GAN/blob/main/CNN_LSTM_MSE_GAN_35Timepoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Author
- **Meng-Hsuan (Michelle) Wu** (JHU)

## Projects
- Least Squares Mean Squared Generative Adversarial Network 35 timesteps training

In [ ]:
import os
import os.path
import cv2
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as fn
import torch.nn as nn
from torch import linalg as LA
from torch import optim
import torch.nn.functional as F
from torch.autograd import grad as torch_grad
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import math
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import Dataset, random_split,TensorDataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import pickle
import shutil
from torchvision.utils import make_grid
import imageio
from google.colab import drive
drive.mount('/content/drive')
from matplotlib import cm
from math import floor
from torch.nn.modules.pooling import MaxPool2d
import random


Mounted at /content/drive


# Loading Datapoint

In [ ]:
num_input_pics = 7
tot_pics_series = 100 #100
num_series = 100
img_channel = 3
img_height = 288
img_width = 432


# num_train + num_val + num_test = num_series
num_train = 80 # 80
num_val = 10
num_test = 10
batch_train_num = 8
batch_valid_num = 10
batch_test_num = 10




In [ ]:
seq_train = torch.arange(0, num_train)
seq_val = torch.arange(num_train, num_train + num_val)
seq_test = torch.arange(100, 110)


train_data = TensorDataset(seq_train)
val_data = TensorDataset(seq_val)
test_data = TensorDataset(seq_test)

train_dl = DataLoader(train_data, batch_size = batch_train_num, shuffle = True)
valid_dl = DataLoader(val_data,   batch_size = batch_valid_num, shuffle = False)
test_dl = DataLoader(test_data,   batch_size = batch_test_num, shuffle = False)

# Model (Generator)

## Encoder

In [ ]:
class ConvNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.network = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size = 2, stride = 1),
        nn.ReLU(),
        nn.BatchNorm2d(64,momentum=0.9),
        nn.MaxPool2d(2),

        nn.Conv2d(64, 128, kernel_size = 3, stride = 1),
        nn.ReLU(),
        nn.BatchNorm2d(128,momentum=0.9),
        nn.MaxPool2d(3),

        nn.Conv2d(128, 256, kernel_size = 2, stride = 1),
        nn.ReLU(),
        nn.BatchNorm2d(256,momentum=0.9),
        nn.MaxPool2d(3),

        nn.Conv2d(256, 256, kernel_size = 2, stride = 1),
        nn.ReLU(),

        nn.BatchNorm2d(256,momentum=0.9),
        nn.MaxPool2d(3),

        nn.Conv2d(256, 512, kernel_size = 3, stride = 1),
        nn.ReLU(),
        nn.Dropout(0.2)
    )

  def forward(self, x):
    cnn_val = self.network(x)
    return_val = F.max_pool2d(cnn_val, kernel_size=cnn_val.size()[2:])
    return_val = torch.squeeze(return_val)
    return return_val

In [ ]:
class Encoder(nn.Module):
  def __init__(self):
    super(Encoder, self).__init__()
    self.convNN = ConvNN()
    self.lstm = nn.LSTM(input_size = 512, hidden_size = 512,
                        batch_first = True, bidirectional = True,
                        num_layers = 4, dropout = 0.2)

  def forward(self, input):
    batch_size = int(input.shape[0]/img_length)
    in_features = self.convNN(input)
    in_features = torch.reshape(in_features,(batch_size, img_length, 512))
    output, (h_n, c_n) = self.lstm(in_features)
    return h_n[-1]

## Decoder

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(in_channels = 512, out_channels = 256,
                               kernel_size = 4, stride = 1, padding = 0,
                               bias=False),
            nn.ReLU(),
            nn.Upsample(scale_factor = 2, mode = 'nearest'),
            nn.BatchNorm2d(256),

            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.ReLU(),
            nn.Upsample(scale_factor = 2, mode = 'nearest'),
            nn.BatchNorm2d(128),

            nn.ConvTranspose2d(128, 128, 4, 2, 1, bias=False),
            nn.ReLU(),
            nn.Upsample(scale_factor = 2, mode = 'nearest'),
            nn.BatchNorm2d(128),

            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.ReLU(),
            nn.Upsample(scale_factor = 2, mode = 'nearest'),
            nn.BatchNorm2d(64),

            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Sigmoid()

        )
    def forward(self, input):
      return self.main(input)

# Discriminator (Critic)

In [ ]:
class Discriminator(nn.Module):
  # img_size = (32, 32, 1)
  # dim = 16 # should be 64
  def __init__(self, img_size, dim):
      super(Discriminator, self).__init__()

      self.img_size = img_size

      self.image_to_features = nn.Sequential(
          nn.Conv2d(self.img_size[2], dim, 4, 2, 1),
          nn.LeakyReLU(0.2),
          nn.Conv2d(dim, 2 * dim, 4, 2, 1),
          nn.LeakyReLU(0.2),
          nn.Conv2d(2 * dim, 4 * dim, 4, 2, 1),
          nn.LeakyReLU(0.2),

          nn.Conv2d(4 * dim, 8 * dim, 4, 2, 1),
          nn.LeakyReLU(0.2),
          nn.Conv2d(8 * dim, self.img_size[2], 4, 2, 1),

          nn.Sigmoid()
      )

      self.features_to_prob = nn.Sequential(
          nn.Linear(192, 1),
          nn.Sigmoid()
      )

  def forward(self, input_data):
    batch_size = input_data.size()[0]
    n_channels = input_data.size()[1]
    x = self.image_to_features(input_data)
    x = x.view(batch_size, -1)
    return self.features_to_prob(x)

# Models

In [ ]:
class CLSTM(nn.Module):
  def __init__(self, latent_dim):
    super(CLSTM, self).__init__()
    self.encoder = Encoder()
    self.decoder = Decoder()
    self.latent_dim = latent_dim
  def forward(self, x):
    y_hidden = self.encoder(x)
    y_hidden = torch.unsqueeze(y_hidden,2)
    y_hidden = torch.unsqueeze(y_hidden,3)
    y_predict = self.decoder(y_hidden)
    return y_predict
  def sample_latent(self, num_samples):
    return torch.randn((num_samples, self.latent_dim))

# Train

In [ ]:
class Trainer():
  def __init__(self, clstm, discriminator, gen_optimizer, dis_optimizer,
               checkpoint_path, backup_model_path, img_channel,img_height,img_width,new_img_height,new_img_width,
               train_mse,val_mse,train_g_loss,train_d_loss,val_g_loss,val_d_loss,
               random_timestep,tot_pics_series,
               num_input_pics,save_model_every = 200, print_every = 10,
               use_cuda=True):

    self.G = clstm

    self.G_opt = gen_optimizer
    self.D = discriminator
    self.D_opt = dis_optimizer

    self.checkpoint_path = checkpoint_path
    self.backup_model_path = backup_model_path

    self.img_channel = img_channel
    self.old_img_height = img_height
    self.old_img_width = img_width
    self.new_img_height = new_img_height
    self.new_img_width = new_img_width

    self.num_input_pics = num_input_pics
    self.tot_pics_series = tot_pics_series
    self.start_time = self.num_input_pics
    self.random_timestep = random_timestep


    self.train_mse = train_mse
    self.val_mse = val_mse
    self.train_g_loss = train_g_loss
    self.val_g_loss = val_g_loss
    self.train_d_loss = train_d_loss
    self.val_d_loss = val_d_loss

    self.temp_train_d_loss = []
    self.temp_train_g_loss = []
    self.temp_val_d_loss = []
    self.temp_val_g_loss = []

    self.temp_train_mse = []
    self.temp_val_mse = []

    self.save_model_every = save_model_every
    self.print_every = print_every

    self.use_cuda = use_cuda



    if self.use_cuda:
        self.G = self.G.cuda()
        self.D = self.D.cuda()


  def _critic_train_iteration(self, y_pred, y_true, train):
    batch_size = y_true.size()[0]
    theoretical_real = torch.ones(batch_size,1) # this is b
    theoretical_fake = torch.zeros(batch_size,1) # this is a
    if self.use_cuda:
      theoretical_real = theoretical_real.cuda()
      theoretical_fake = theoretical_fake.cuda()

    # Calculate probabilities on real and generated data
    d_real = self.D(y_true)
    d_generated = self.D(y_pred)

    # Create total loss and optimize
    if train:
      self.D_opt.zero_grad()
      d_real = d_real.view(batch_size, -1)
      d_generated = d_generated.view(batch_size,-1)
      real_loss = adversarial_loss(d_real, theoretical_real)
      fake_loss = adversarial_loss(d_generated, theoretical_fake)
      d_loss = (1/2)* (real_loss + fake_loss)

      d_loss.backward(retain_graph = True)
      self.D_opt.step()
      self.temp_train_d_loss.append(d_loss.data.cpu().numpy())
    else:
      d_real = d_real.view(batch_size, -1)
      d_generated = d_generated.view(batch_size,-1)
      real_loss = adversarial_loss(d_real, theoretical_real)
      fake_loss = adversarial_loss(d_generated, theoretical_fake)
      d_loss = (1/2)* (real_loss + fake_loss)
      self.temp_val_d_loss.append(d_loss.data.cpu().numpy())
  def _generator_train_iteration(self, y_pred, y_true, train):
    batch_size = y_true.size()[0]
    theoretical_real = torch.ones(batch_size,1)
    pixel_diff = adversarial_loss(y_true,y_pred)
    if self.use_cuda:
      theoretical_real = theoretical_real.cuda()
    if train:
      self.G_opt.zero_grad()
      d_generated = self.D(y_pred)
      g_loss = 0.5*(adversarial_loss(d_generated,theoretical_real) + pixel_diff)
      g_loss.backward()
      self.G_opt.step()

      # Record loss
      g_loss = g_loss.data.cpu().numpy()
      self.temp_train_g_loss.append(g_loss)
      self.temp_train_mse.append(pixel_diff)
    else:
      d_generated = self.D(y_pred)
      g_loss = 0.5*(adversarial_loss(d_generated, theoretical_real) + pixel_diff)

      # Record loss
      g_loss = g_loss.data.cpu().numpy()
      self.temp_val_g_loss.append(g_loss)
      self.temp_val_mse.append(pixel_diff)
  def _train_epoch(self,train_loader, train_data):

    self.G.train()

    train_batch_num = 0
    for batch_train in train_loader:
      curr_batch_train = train_data[train_batch_num]
      train_batch_num +=1
      for t in range(len(self.random_timestep)):
        curr_y_time = self.random_timestep[t]
        curr_x_start_time = curr_y_time-self.num_input_pics
        x_train = curr_batch_train[:, curr_x_start_time: curr_y_time,:,:,: ]
        y_train = curr_batch_train[:, curr_y_time, :,:,:]


        if self.use_cuda:
          x_train, y_train = x_train.cuda(), y_train.cuda()
        x_train = fn.center_crop(x_train,output_size = [256])

        n_series = x_train.shape[0]

        x_train_new_dim = (n_series*self.num_input_pics, self.img_channel, self.new_img_height, self.new_img_width)
        x_train_new = torch.reshape(x_train,x_train_new_dim)
        y_train_predict = self.G(x_train_new)
        y_train_crop = fn.center_crop(y_train, output_size=[256])
        y_train_predict_crop = fn.center_crop(y_train_predict, output_size = [256])
        self._critic_train_iteration(y_train_predict_crop, y_train_crop, train = True)
        self._generator_train_iteration(y_train_predict_crop, y_train_crop, train = True)


  def _validate(self, val_loader, val_data):

    self.G.eval()

    with torch.no_grad():
      val_batch_num = 0

      for batch_val in val_loader:
        curr_batch_val = val_data[val_batch_num]
        val_batch_num +=1
        for t in range(len(self.random_timestep)):
          curr_y_time = self.random_timestep[t]
          curr_x_start_time = curr_y_time-self.num_input_pics
          x_val = curr_batch_val[:, curr_x_start_time: curr_y_time,:,:,: ]
          y_val = curr_batch_val[:, curr_y_time, :,:,:]
          if self.use_cuda:
            x_val, y_val = x_val.cuda(), y_val.cuda()
          x_val = fn.center_crop(x_val,output_size = [256])

          n_series = x_val.shape[0]

          x_val_new_dim = (n_series*self.num_input_pics, self.img_channel, self.new_img_height, self.new_img_width)
          x_val_new = torch.reshape(x_val,x_val_new_dim)
          y_val_predict = self.G(x_val_new)
          y_val_crop = fn.center_crop(y_val, output_size=[256])
          y_val_predict_crop = fn.center_crop(y_val_predict, output_size = [256])
          self._critic_train_iteration(y_val_predict_crop, y_val_crop, train = False)
          self._generator_train_iteration(y_val_predict_crop, y_val_crop, train = False)


  def _data_generator(self, y_index,num_series):
    data = np.zeros(shape = (num_series, self.tot_pics_series, self.img_channel, self.old_img_height, self.old_img_width))
    for y_idx in range(num_series):
      model_path = "/content/drive/MyDrive/Capstone/Capstone_data_jpg/"+str(y_index[y_idx])
      for entry in range(self.tot_pics_series):
        img_path = model_path+'/'+"{:02d}".format(entry)+'im.jpg'
        x_to_convert = Image.open(img_path)
        x = np.asarray(x_to_convert)
        x = np.moveaxis(x,-1,0) # change image dimension to channel first
        data[y_idx][entry] = x
    data = torch.tensor(np.array(data/255).astype(np.float32))
    return data

  def train(self, train_loader, start_epochs, n_epochs, validate_loader):
    train_data = {}
    val_data = {}
    for epoch in range(start_epochs, n_epochs):

      # load the data at the first epoch
      if epoch == start_epochs:
        val_batch_num = 0
        train_batch_num = 0

        for batch_train in train_loader:
          y_idx = batch_train[0].tolist()
          num_series = len(y_idx)
          data_train = self._data_generator(y_idx, num_series)
          train_data[train_batch_num] = data_train
          train_batch_num +=1

        for batch_val in validate_loader:
          y_idx = batch_val[0].tolist()
          num_series = len(y_idx)
          data_val = self._data_generator(y_idx, num_series)
          val_data[val_batch_num] = data_val
          val_batch_num +=1
        print("data complete!")

      self._train_epoch(train_loader,train_data)
      self._validate(validate_loader,val_data)

      avg_train_g_loss = sum(self.temp_train_g_loss) / len(self.temp_train_g_loss)
      avg_train_d_loss = sum(self.temp_train_d_loss) / len(self.temp_train_d_loss)
      avg_val_g_loss = sum(self.temp_val_g_loss) / len(self.temp_val_g_loss)
      avg_val_d_loss = sum(self.temp_val_d_loss) / len(self.temp_val_d_loss)
      avg_train_mse = sum(self.temp_train_mse)/len(self.temp_train_mse)
      avg_val_mse = sum(self.temp_val_mse)/len(self.temp_val_mse)

      if epoch % self.print_every == 0:
        print("Iteration {}".format(epoch + 1))
        print("D: {}".format(avg_train_d_loss))
        print("val_D: {}".format(avg_val_d_loss))
        print("G: {}".format(avg_train_d_loss))
        print("val_G: {}".format(avg_val_g_loss))
        print("train_mse:{}".format(avg_train_mse))
        print("val_mse:{}".format(avg_val_mse))

      ##############
      # checkpoint #
      ##############
      if self.use_cuda:
        self.G, self.D,avg_train_mse, avg_val_mse = self.G.cpu(), self.D.cpu(), avg_train_mse.cpu().detach().numpy(), avg_val_mse.cpu().detach().numpy()
      self.train_g_loss.append(avg_train_g_loss)
      self.val_g_loss.append(avg_val_g_loss)
      self.train_d_loss.append(avg_train_d_loss)
      self.val_d_loss.append(avg_val_d_loss)
      self.train_mse.append(avg_train_mse)
      self.val_mse.append(avg_val_mse)
      checkpoint = {
        'epoch': epoch + 1,
        'generator_state_dict': self.G.state_dict(),
        'generator_optimizer': self.G_opt.state_dict(),
        'discriminator_state_dict': self.D.state_dict(),
        'discriminator_optimizer': self.D_opt.state_dict(),
        'train_mse': self.train_mse,
        'val_mse': self.val_mse,
        'train_g_loss':self.train_g_loss,
        'val_g_loss': self.val_g_loss,
        'train_d_loss':self.train_d_loss,
        'val_d_loss':self.val_d_loss
        }
      # save checkpoint
      save_ckp(checkpoint, False, self.checkpoint_path)

      # save another copy in case model corrupt
      if epoch % self.save_model_every == 0:
        backup_path = self.backup_model_path + "/"+ "16_clstm_fix_lsgan_random_choice"+str(epoch)+".pt"
        save_ckp(checkpoint,False , backup_path)
      self.temp_train_d_loss = []
      self.temp_train_g_loss = []
      self.temp_val_d_loss = []
      self.temp_val_g_loss = []
      self.temp_train_mse = []
      self.temp_val_mse = []
      if self.use_cuda:
        self.G, self.D = self.G.cuda(), self.D.cuda()


# Checkpoint

In [ ]:
def save_ckp(state, is_best, checkpoint_path):
  f_path = checkpoint_path
  torch.save(state, f_path)

In [ ]:
def load_ckp(checkpoint_fpath, gen_model, gen_opt, discrim_model, discrim_opt):
    device = torch.device("cuda")
    # load check point
    checkpoint = torch.load(checkpoint_fpath,map_location="cuda:0")
    # initialize state_dict from checkpoint to model
    gen_model.load_state_dict(checkpoint['generator_state_dict'])
    discrim_model.load_state_dict(checkpoint['discriminator_state_dict'])
    gen_model = gen_model.to(device)
    discrim_model = discrim_model.to(device)
    # initialize optimizer from checkpoint to optimizer
    gen_opt.load_state_dict(checkpoint['generator_optimizer'])
    discrim_opt.load_state_dict(checkpoint['discriminator_optimizer'])

    # mse
    train_mse = checkpoint['train_mse']
    val_mse = checkpoint['val_mse']

    # other loss
    train_g_loss = checkpoint['train_g_loss']
    train_d_loss = checkpoint['train_d_loss']
    val_g_loss = checkpoint['val_g_loss']
    val_d_loss = checkpoint['val_d_loss']


    # return model, optimizer, epoch value, min validation loss
    return checkpoint['epoch'], gen_model, discrim_model, gen_opt, discrim_opt, train_mse, val_mse, train_g_loss, train_d_loss, val_g_loss, val_d_loss


# Setting Parameters

In [ ]:
# Train model
# setting parameters

num_input_pics = 7
tot_pics_series = 100


start_epoch = 0
n_epochs = 1000

img_channel = 3
img_height = 288
img_width = 432

new_img_height = 256
new_img_width = 256

random.seed(1)
random_timestep = [num_input_pics]

random_timestep_num = 35
numbers = random.sample(range(num_input_pics+1, tot_pics_series), random_timestep_num-1)
random_timestep.extend(numbers)
print(random_timestep)
print(len(random_timestep))

img_length = num_input_pics
clstm = CLSTM(512)

img_size = (288, 432, 3)
lr = 0.00001
discriminator = Discriminator(img_size=img_size, dim=16)
g_optimizer = optim.RMSprop(clstm.parameters(), lr = lr)
d_optimizer = optim.Adam(discriminator.parameters(), lr = lr)

checkpoint_path = "/content/drive/MyDrive/Capstone/Capstone_models/16_clstm_fix_lsgan_random_choice.pt"
backup_model_path = "/content/drive/MyDrive/Capstone/Capstone_checkpoint_models"



train_mse = []
val_mse = []
train_g_loss = []
val_g_loss = []
train_d_loss = []
val_d_loss = []

print(clstm)
print(discriminator)

save_model_every = 100

print_every = 10

adversarial_loss = torch.nn.MSELoss()
use_cuda = torch.cuda.is_available()

trainer = Trainer(clstm, discriminator, g_optimizer, d_optimizer, checkpoint_path,
                  backup_model_path, img_channel, img_height, img_width, new_img_height, new_img_width,
                  train_mse, val_mse, train_g_loss, train_d_loss, val_g_loss, val_d_loss, random_timestep,tot_pics_series,
                  num_input_pics, save_model_every, print_every, use_cuda=torch.cuda.is_available())

[7, 25, 80, 16, 40, 23, 71, 65, 68, 91, 56, 34, 20, 70, 11, 57, 63, 8, 93, 42, 37, 21, 48, 86, 10, 77, 9, 90, 35, 62, 54, 74, 41, 22, 73]
35
CLSTM(
  (encoder): Encoder(
    (convNN): ConvNN(
      (network): Sequential(
        (0): Conv2d(3, 64, kernel_size=(2, 2), stride=(1, 1))
        (1): ReLU()
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
        (5): ReLU()
        (6): BatchNorm2d(128, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        (7): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
        (8): Conv2d(128, 256, kernel_size=(2, 2), stride=(1, 1))
        (9): ReLU()
        (10): BatchNorm2d(256, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        (11): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1

In [ ]:

# trainer.train(train_dl, start_epoch,  n_epochs, valid_dl)

# Load Model

In [ ]:
# loading checkpoint if model fail
ckp_path = "/content/drive/MyDrive/Capstone/Capstone_checkpoint_models/16_clstm_fix_lsgan_random_choice500.pt"
backup_model_path = "/content/drive/MyDrive/Capstone/Capstone_checkpoint_models"
clstm = CLSTM(512)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda")

img_size = (288, 432, 3)
discriminator = Discriminator(img_size=img_size, dim=16)

# Initialize optimizers
lr = 0.00001
adversarial_loss = torch.nn.MSELoss()
g_optimizer = optim.RMSprop(clstm.parameters(), lr = lr)
d_optimizer = optim.Adam(discriminator.parameters(), lr = lr)
use_cuda = torch.cuda.is_available()


batch_train_num = 8
batch_valid_num = 10
batch_test_num= 10

img_length = num_input_pics

start_epoch, g_model, d_model, g_opt, d_opt,train_mse, val_mse,train_g_loss, train_d_loss, val_g_loss, val_d_loss= load_ckp(ckp_path, clstm, g_optimizer,discriminator,d_optimizer)
print(g_model)


CLSTM(
  (encoder): Encoder(
    (convNN): ConvNN(
      (network): Sequential(
        (0): Conv2d(3, 64, kernel_size=(2, 2), stride=(1, 1))
        (1): ReLU()
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
        (5): ReLU()
        (6): BatchNorm2d(128, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        (7): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
        (8): Conv2d(128, 256, kernel_size=(2, 2), stride=(1, 1))
        (9): ReLU()
        (10): BatchNorm2d(256, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        (11): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
        (12): Conv2d(256, 256, kernel_size=(2, 2), stride=(1, 1))
        (13): ReLU()
        (14): BatchNorm2d(256, eps=

In [ ]:
trainer = Trainer(clstm, discriminator, g_optimizer, d_optimizer, ckp_path,
                  backup_model_path, img_channel, img_height, img_width, new_img_height, new_img_width,
                  train_mse, val_mse,train_g_loss, train_d_loss, val_g_loss, val_d_loss, random_timestep,tot_pics_series,
                  num_input_pics, save_model_every=100, print_every=10, use_cuda=torch.cuda.is_available())



In [ ]:
# continue training
#trainer.train(train_dl, start_epoch,  n_epochs, valid_dl)